
# Sprawozdanie z laboratorium - sieci neuronowe


Temat: Speech to text

Skład grupy: Sebastian Krakowiak, Jarosław Kuczyński

## **Opis Zadania**

Nasze zadanie polega na stworzeniu programu, opartego na sieciach neuronowych, do przetwarzania mowy w tekst.

## **Dane**

Nasza paczka danych to zbiór 38 GB, składający się z kilkusekundowych nagrań w języku angielskim, zapisanych w formacie mp3. 

### **Źródła danych**

Dane, które użyliśmy są zbiorem danych mowy open source, pobranym ze strony programu
[Mozilla Common Voice](https://voice.mozilla.org/pl/datasets).

### **Analiza danych**

Dane są zróżnicowane ze względu na akcent, płeć, oraz wiek. Akcenty, które wchodzą w skład zbioru:
* 23% United States English
* 9% England English
* 4% India and South Asia (India, Pakistan, Sri Lanka)
* 3% Australian English
* 3% Canadian English
* 2% Scottish English
* 1% Irish English
* 1% Southern African (South Africa, Zimbabwe, Namibia)
* 1% New Zealand English

Grupy wiekowe są podzielone wg. następujących proporcji:
* 22% 19 - 29
* 15% 30 - 39
* 9% 40 - 49
* 5% 50 - 59
* 5% < 19
* 4% 60 - 69
* 1% 70 - 79

Podział ze względu na płeć wygląda następująco:
* 46% mężczyzna
* 13% kobieta

## **Model**

### **Opis**

System został zbudowany przy pomocy następujących narzędzi:
* Python - Język programowania wysokiego poziomu ogólnego przeznaczenia.
* Numpy - Biblioteka do obliczeń naukowych.
* Keras - Biblioteka przeznaczona do sieci neuronowych.
* Librosa - Biblioteka przetwarzająca dźwięk.

Wbrew powszechnemu przekonaniu o wykorzstaniu rekurencyjnych sieci neuronowych w zadaniach sekwencyjnych (do których zaliczamy właśnie przetwarzanie, czy rozpoznawanie mowy), wykorzystujemy konwolucyjne sieci neuronowe. 

Jako uzasadnienie naszej decyzji przymujemy przewagę wydajnościową sieci konwolucyjnych. Sieci rekurencyjne przetwarzają informację sekwencyjnie, gdzie dane są zależne od siebie, wykonywane są takie same operacje na każdym jednym elemencie. Sprawia to, że każdy kolejny krok jest uzależniony od poprzedniego, co sprawia, że obliczenia węzłów nie mogą być podzielone na osobne rdzenie, wszystkie muszą być wykonane w odpowiedniej kolejności. Sprawia to, że trening sieci rekurencyjnych jest wolna i nie pozwala na wykorzystanie maximum wydajności sprzętu.

Sieci konwolucyjne są pozbawione tych ograniczeń, na dodatek wykorzystując je na zasadzie encodera, możemy z powodzeniem użyć ich w zadaniu typu speech-to-text.

### **Działanie**

Zaczynamy od załadowania potrzebnych nam bibliotek.
```
import os, sys
import librosa as lib
import IPython.display as ipd
import matplotlib.pyplot as plt
import numpy as np
from scipy.io import wavfile #for audio processing
import keras
from keras.models import load_model
from keras.layers import Dense, Dropout, Flatten, Conv1D, Input, MaxPooling1D
from keras.models import Model
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend as K
K.clear_session()
```

W kolejnym kroku wczytujemy dane, następnie przetwarzamy je za pomocą biblioteki Librosa na wykres częstotliwości dźwięku.
```
train_audio_path = "F:\\projekt py\\wav"
files = lib.util.find_files(train_audio_path)
(samples, sample_rate) = lib.load(train_audio_path+'\\common_voice_en_1.wav', sr=16000)
amount_of_samples = len(samples)
time_array = np.arange(0, float(amount_of_samples), 1) / sample_rate
scaled_samples = samples / (2 ** 15)
fig = plt.figure(figsize=(14, 8))
ax1 = fig.add_subplot(211)
ax1.set_title('trenowany dźwięk')
ax1.set_xlabel('time')
ax1.set_ylabel('Amplitude')
ax1.plot(time_array, scaled_samples, linewidth=0.3, alpha=0.7, color='#004bc6')
plt.show()
 ```
![wykres](https://doc-14-3o-docs.googleusercontent.com/docs/securesc/lbv1c7hanio529dnbtu5j1g7gtjunqtj/4sq7q7a9lnp18pksmii6b41gke7jds96/1579651200000/00782543097958797517/00782543097958797517/1MAVMPdz5IKk-vUMk-OaQJCpGkD-Wr_uh?e=view&authuser=0&nonce=8jtkpvilg5aos&user=00782543097958797517&hash=0mauh0dffggvobe9p3h8ub24oreutq66)

Następnie resamplujemy nasze nagrania na optymalne 8000Hz


```
samples = librosa.resample(samples, sample_rate, 8000)
ipd.Audio(samples, rate=8000)
```

Budujemy naszą architekturę używając conv1d. Jest to konwolucyjna sieć neuronowa, która działa tylko w jednym wymiarze.

![Przybliżona architektura](https://s3-ap-south-1.amazonaws.com/av-blog-media/wp-content/uploads/2019/07/model_architecture.jpg.jpg)

Implementujemy model przy użyciu biblioteki keras
```
inputs = Input(shape=(8000,1))

conv = Conv1D(8,13, padding='valid', activation='relu', strides=1)(inputs)
conv = MaxPooling1D(3)(conv)
conv = Dropout(0.3)(conv)


conv = Conv1D(16, 11, padding='valid', activation='relu', strides=1)(conv)
conv = MaxPooling1D(3)(conv)
conv = Dropout(0.3)(conv)

conv = Conv1D(32, 9, padding='valid', activation='relu', strides=1)(conv)
conv = MaxPooling1D(3)(conv)
conv = Dropout(0.3)(conv)

conv = Conv1D(64, 7, padding='valid', activation='relu', strides=1)(conv)
conv = MaxPooling1D(3)(conv)
conv = Dropout(0.3)(conv)

conv = Flatten()(conv)

conv = Dense(256, activation='relu')(conv)
conv = Dropout(0.3)(conv)

conv = Dense(128, activation='relu')(conv)
conv = Dropout(0.3)(conv)

outputs = Dense(len(labels), activation='softmax')(conv)

model = Model(inputs, outputs)
model.summary()
```
Definiujemy funckję strat
```
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
```
Tworzymy wczesne zatrzymanie, oraz checkpoint-y, aby zatrzymać sieć w odpowiednim momencie i zatrzymać najlepszy model.
```
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10, min_delta=0.0001) 
mc = ModelCheckpoint('best_model.hdf5', monitor='val_acc', verbose=1, save_best_only=True, mode='max')
```
Trenujemy model na batchu wielkości 32
```
history=model.fit(x_tr, y_tr ,epochs=100, callbacks=[es,mc], batch_size=32, validation_data=(x_val,y_val))
```
Ładujemy najlepszy model
```
from keras.models import load_model
model=load_model('best_model.hdf5')
```
Definiujemy funkcję, która przewiduje tekst dla danego audio
```
def predict(audio):
    prob=model.predict(audio.reshape(1,8000,1))
    index=np.argmax(prob[0])
    return classes[index]
```
Convertujemy pliki audio na tekst
```
os.listdir('F:/projekt py/test')
filepath='F:/projekt py/test/'

#reading the voice commands
samples, sample_rate = librosa.load(filepath + '/' + 'command_voice_en_493126.wav.wav', sr = 16000)
samples = librosa.resample(samples, sample_rate, 8000)
ipd.Audio(samples,rate=8000)  

predict(samples)
```